# This is a notebook for the Coursera IBM Certification Capstone project 

In [8]:
# Import the modules to be used 
import pandas as pd 
import numpy as np 
import requests as rq
from bs4 import BeautifulSoup as bs

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## 1. Scrapping the neighborhoods in the city of Toronto Data from Wikipedia

In [9]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=rq.get(url)
print(page.status_code)

200


In [11]:
page_soup=bs(page.text,'html.parser')

In [19]:
A=[]
B=[]
C=[]

for i in page_soup.find_all(class_="wikitable sortable"):
    for item in i.findAll('tr'):
        entry=item.findAll('td')
        if len(entry)==3:
            A.append(entry[0].find(text=True))
            B.append(entry[1].find(text=True))
            C.append(entry[2].find(text=True))


In [24]:
df_ne=pd.DataFrame({'PostalCode':A,
                   'Borough':B,
                   'Neighborhood':C})
df_ne.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [284]:
### wrangle the data 
# replace the new line 
df_ne['PostalCode']=df_ne['PostalCode'].map(lambda x: x.replace("\n"," "))
df_ne['Borough'] = df_ne['Borough'].map(lambda x: x.replace("\n"," "))
df_ne['Neighborhood'] = df_ne['Neighborhood'].map(lambda x: x.replace("\n"," "))
# perhaps we would want to remove the entries that have not been assigned 

df_ne = df_ne.drop(df_ne[df_ne.Borough == "Not assigned"].index)
df_ne.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [53]:
df_ne.shape

(103, 3)

#### 1.1 getting  longitude coordinates of each neighborhood.

In [56]:
import geocoder

In [122]:
# !pip install  geocoder

In [123]:

long_l = []
lat_l = []
for i in df_ne['PostalCode']:
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(i))
        lat_lng_coords = g.latlng
    lat_l.append(lat_lng_coords[0])
    long_l.append(lat_lng_coords[1])

In [58]:

print(len(long_l),len(lat_l))

103 103


In [73]:
## Get them to a combined_data_frame
df_long=pd.DataFrame({"Longitude":long_l})
df_lat=pd.DataFrame({"Latitude":lat_l})

df_ne_loc=pd.concat([df_ne,df_lat,df_long],axis=1).dropna()

location_data=pd.read_csv('http://cocl.us/Geospatial_data')

In [74]:
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [283]:
# location_data.rename(columns={"Postal Code":"PostalCode"},inplace=True)

# location_data.columns=["PostalCode", "Latitude", "Longitude"]

# new.head()

# df_ne.columns=["PostalCode", "Borough", "Neighborhood"]

# new=df_ne.merge(location_data,on='PostalCode')

# pd.merge(df_ne,location_data,on='PostalCode')


# 2. K means

In [85]:
location_data.columns
df_ne.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [75]:
# import the libraries to use 
from pandas.io.json import json_normalize 
import matplotlib.pyplot as plt
import folium
from sklearn.cluster import KMeans


##### Select a specific venue category

In [285]:
df_ne_loc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.65514,-79.36265
3,M4A,North York,Victoria Village,43.72321,-79.45141
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66449,-79.39302
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.66277,-79.52831
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81153,-79.19552


In [142]:
neighborhood_lat = df_ne_loc.loc[11, 'Latitude'] # neighborhood latitude value
neighborhood_long =df_ne_loc.loc[11, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_ne_loc.loc[11, 'Neighborhood'] # neighborhood name

print('Latitude and longitude coordinates  of\n {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_lat, 
                                                               neighborhood_long))

Latitude and longitude coordinates  of
 Don Mills  are 43.65279000000004, -79.55405999999994.


In [143]:
## making api calls to foursquare
CLIENT_ID = 'DPIZ13PIHP1BYV3EAJVTRO4GPMFPOCBVTHKT45OPNYCKCWGS' # my Foursquare ID
CLIENT_SECRET = 'MMKATV3MMUCFBLIQ4MJELX5ZDD12AFCMCHKJWNI223NRHCIL' #my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: DPIZ13PIHP1BYV3EAJVTRO4GPMFPOCBVTHKT45OPNYCKCWGS
CLIENT_SECRET:MMKATV3MMUCFBLIQ4MJELX5ZDD12AFCMCHKJWNI223NRHCIL


In [164]:
# latitude = neighborhood_lat
# longitude = neighborhood_long
VERSION = '20180605' 
near="coffee"
search_query = neighborhood_name
radius = 5000
LIMIT = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&section={}&near={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION,near,search_query, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=DPIZ13PIHP1BYV3EAJVTRO4GPMFPOCBVTHKT45OPNYCKCWGS&client_secret=MMKATV3MMUCFBLIQ4MJELX5ZDD12AFCMCHKJWNI223NRHCIL&v=20180605&section=coffee&near=Don Mills &radius=5000&limit=500


### Send a get request and see the results

In [195]:
results = rq.get(url).json()
# results

## get the infomation parts  that we need about the location of interest


In [168]:
# assign relevant part of JSON to venues
venues = results['response']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,geocode.cc,geocode.center.lat,geocode.center.lng,geocode.displayString,geocode.geometry.bounds.ne.lat,geocode.geometry.bounds.ne.lng,geocode.geometry.bounds.sw.lat,geocode.geometry.bounds.sw.lng,geocode.longId,geocode.what,...,headerLocation,headerLocationGranularity,query,suggestedBounds.ne.lat,suggestedBounds.ne.lng,suggestedBounds.sw.lat,suggestedBounds.sw.lng,suggestedFilters.filters,suggestedFilters.header,totalResults
0,CA,43.73341,-79.3329,"Don Mills, ON, Canada",43.792347,-79.28569,43.703835,-79.365639,72057594043869538,,...,Don Mills,city,coffee,43.781575,-79.272274,43.684413,-79.390111,"[{'name': 'Open now', 'key': 'openNow'}]",Tap to show:,53


In [194]:
# venues.keys() The results we want are contained in the groups
# ## let us explore further the shops around the city 
# venues["headerLocationGranularity"],venues["headerLocation"],venues["headerFullLocation"]

# venues['geocode']


# print(venues["groups"][0]["type"],venues["groups"][0]["name"])


# items = venues["groups"][0]["items"]
# print("number of items: %i" % len(items))
# items[5]

# Looping through the list 

# items[5]['venue']

In [186]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [187]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks Reserve Bar,Coffee Shop,43.735764,-79.344156
1,Aroma Espresso Bar,Café,43.734400,-79.345204
2,Ice 'n Cake,Ice Cream Shop,43.734292,-79.345198
3,Panera Bread,Bakery,43.734829,-79.343888
4,Baretto Caffé,Café,43.744456,-79.346460


## Explore neaby

In [217]:

def getNearbyVenues(names, latitudes, longitudes, radius=400):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [221]:
radius = 400
LIMIT =100
toronto_venues = getNearbyVenues(names=df_ne_loc['Neighborhood'],
                                   latitudes=df_ne_loc['Latitude'],
                                   longitudes=df_ne_loc['Longitude'])

Parkwoods 
Victoria Village 
Regent Park, Harbourfront 
Lawrence Manor, Lawrence Heights 
Queen's Park, Ontario Provincial Government 
Islington Avenue, Humber Valley Village 
Malvern, Rouge 
Don Mills 
Parkview Hill, Woodbine Gardens 
Garden District, Ryerson 
Glencairn 
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale 
Rouge Hill, Port Union, Highland Creek 
Don Mills 
Woodbine Heights 
St. James Town 
Humewood-Cedarvale 
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood 
Guildwood, Morningside, West Hill 
The Beaches 
Berczy Park 
Caledonia-Fairbanks 
Woburn 
Leaside 
Central Bay Street 
Christie 
Cedarbrae 
Hillcrest Village 
Bathurst Manor, Wilson Heights, Downsview North 
Thorncliffe Park 
Richmond, Adelaide, King 
Dufferin, Dovercourt Village 
Scarborough Village 
Fairview, Henry Farm, Oriole 
Northwood Park, York University 
East Toronto, Broadview North (Old East York) 
Harbourfront East, Union Station, Toronto Islands 
Little Portugal, Trinit

In [225]:
print(toronto_venues.shape)
toronto_venues.tail()

(393, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
388,Davisville North,43.62463,-79.52835,Xtreme Couture,43.624799,-79.528626,Gym / Fitness Center
389,Davisville North,43.62463,-79.52835,Tactical Products Canada,43.626801,-79.529388,Miscellaneous Shop
390,Davisville North,43.62463,-79.52835,Burrito Boyz,43.626657,-79.526349,Burrito Place
391,Davisville North,43.62463,-79.52835,Kingsway Boxing Club,43.627254,-79.526684,Gym
392,Davisville North,43.62463,-79.52835,Sleep Country,43.621340,-79.526708,Mattress Store


In [233]:
toronto_venues.groupby('Neighborhood')['Venue'].count().to_frame().sort_values('Venue',ascending=False).head(10)

,Venue
Neighborhood,
Downsview,30
"Bathurst Manor, Wilson Heights, Downsview North",10
Parkwoods,10
"Islington Avenue, Humber Valley Village",10
"India Bazaar, The Beaches West",10
Humewood-Cedarvale,10
"Humberlea, Emery",10
Humber Summit,10
Scarborough Village,10


## How many unique venue categories are there

In [236]:

print('There are {} uniques categories for the nearby loc .'.format(len(toronto_venues['Venue Category'].unique())))

There are 137 uniques categories for the nearby loc .


### To Cluster 

In [242]:
toronto_dummy = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") # do a one hot encodin
toronto_dummy['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_dummy.columns[-1]] + list(toronto_dummy.columns[:-1])
toronto_dummy = toronto_dummy[fixed_columns]

df_toronto_group=toronto_dummy.groupby('Neighborhood').mean().reset_index()
df_toronto_group.head()

,Neighborhood,Yoga Studio,Airport,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Shop
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Caledonia-Fairbanks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [243]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Find the most common venues in the neigbourhood

In [244]:

num_top_venues = 15

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_toronto_group['Neighborhood']

for ind in np.arange(df_toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_toronto_group.iloc[ind, :], num_top_venues)

# neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Fish & Chips Shop,Park,Pet Store,Gym,Burrito Place,Sushi Restaurant,Pub,Italian Restaurant,Ice Cream Shop,Fast Food Restaurant,Comic Shop,Convenience Store,Construction & Landscaping,Concert Hall,Wine Shop
1,Bayview Village,Diner,Pharmacy,Pizza Place,Fried Chicken Joint,Convenience Store,Bank,Concert Hall,Dessert Shop,Department Store,Cosmetics Shop,Construction & Landscaping,Wine Shop,Comic Shop,Discount Store,College Theater
2,Berczy Park,Park,Brazilian Restaurant,Grocery Store,Pharmacy,Café,Bus Line,Bakery,Bank,Construction & Landscaping,Dessert Shop,Department Store,Cosmetics Shop,Convenience Store,Wine Shop,Diner
3,"Birch Cliff, Cliffside West",Chinese Restaurant,Grocery Store,Fast Food Restaurant,Bank,Sandwich Place,Pharmacy,Coffee Shop,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Diner,Concert Hall,Construction & Landscaping,Convenience Store
4,Caledonia-Fairbanks,Playground,Middle Eastern Restaurant,Restaurant,Wine Shop,Concert Hall,Department Store,Cosmetics Shop,Convenience Store,Construction & Landscaping,Comic Shop,Diner,Comfort Food Restaurant,College Theater,College Stadium,College Rec Center


In [248]:
## cluster 
kclusters = 7

toronto_clustering=df_toronto_group.drop('Neighborhood',1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

kmeans.labels_[0:10]

array([3, 3, 3, 3, 0, 0, 3, 0, 0, 3])

In [256]:

# neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
toronto_df_cluster = df_ne_loc
toronto_df_cluster= toronto_df_cluster.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_df_cluster.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,M3A,North York,Parkwoods,43.65514,-79.36265,0.0,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Pub,...,Bakery,Event Space,Coffee Shop,Theater,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall
3,M4A,North York,Victoria Village,43.72321,-79.45141,0.0,Clothing Store,Electronics Store,Men's Store,Cosmetics Shop,...,Shopping Mall,American Restaurant,Toy / Game Store,Furniture / Home Store,Diner,Coffee Shop,College Rec Center,College Stadium,College Theater,Comfort Food Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66449,-79.39302,3.0,Park,Monument / Landmark,Art Museum,Theater,...,Concert Hall,Dessert Shop,Department Store,Cosmetics Shop,Convenience Store,Construction & Landscaping,Wine Shop,Comic Shop,Discount Store,Comfort Food Restaurant
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.66277,-79.52831,3.0,Pharmacy,Park,Shopping Mall,Grocery Store,...,Skating Rink,Bank,Wine Shop,Concert Hall,Department Store,Cosmetics Shop,Convenience Store,Construction & Landscaping,Comfort Food Restaurant,Comic Shop
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.81153,-79.19552,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [270]:
toronto_df_cluster=toronto_df_cluster.dropna()

## Mapping the clusters

In [271]:
map_toronto_cluster = folium.Map(location=[latitude,longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []

for lat, lon, poi, cluster in zip(toronto_df_cluster['Latitude'], toronto_df_cluster['Longitude'],toronto_df_cluster['Neighborhood'],toronto_df_cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup=label,
        color=rainbow[(int(cluster)+1)-1],
        fill = True,
        fill_color=rainbow[(int(cluster)+1)-1],
        fill_opacity=0.7).add_to(map_toronto_cluster )
    
map_toronto_cluster 